In [2]:
import os
import pandas as pd
import altair as alt

In [3]:
os.listdir(os.path.join("..", "data", "playstation"))

['ps4_ps5_up_to_FY2023.csv']

In [4]:
filename = "ps4_ps5_up_to_FY2023.csv"

In [5]:
df = pd.read_csv(os.path.join("..", 'data', "playstation", filename))
df

FileNotFoundError: [Errno 2] No such file or directory: '..\\data\\nintendo\\ps4_ps5_up_to_FY2023.csv'

In [ ]:
df.iloc[:, 3:] = df.iloc[:, 3:].fillna("0").astype("string").apply(lambda x: x.str.replace(',', '').apply(lambda x: x.split(".")[0]).astype(int))
df

,Product,Group,Geo,FY3/2017,FY3/2018,FY3/2019,FY3/2020,FY3/2021,FY3/2022,FY3/2023
0,Nintendo Switch,Standard,Japan,60.0,378,385,321,478,256,95
1,Nintendo Switch,Standard,The Americas,120.0,594,688,578,707,522,267
2,Nintendo Switch,Standard,Europe,76.0,379,455,400,502,404,211
3,Nintendo Switch,Standard,Other,18.0,154,169,185,345,174,40
4,Nintendo Switch – OLED Model,OLED,Japan,0.0,0,0,0,0,161,284
5,Nintendo Switch – OLED Model,OLED,The Americas,0.0,0,0,0,0,194,288
6,Nintendo Switch – OLED Model,OLED,Europe,0.0,0,0,0,0,112,238
7,Nintendo Switch – OLED Model,OLED,Other,0.0,0,0,0,0,113,112
8,Nintendo Switch Lite,Lite,Japan,0.0,0,0,200,181,102,58
9,Nintendo Switch Lite,Lite,The Americas,0.0,0,0,233,409,159,142


In [ ]:
index_cols = ["Product", "Group", "Geo"]
df = df.melt(id_vars=index_cols, var_name="Date", value_name="Qty")
df

,Product,Group,Geo,Date,Qty
0,Nintendo Switch,Standard,Japan,FY3/2017,60.0
1,Nintendo Switch,Standard,The Americas,FY3/2017,120.0
2,Nintendo Switch,Standard,Europe,FY3/2017,76.0
3,Nintendo Switch,Standard,Other,FY3/2017,18.0
4,Nintendo Switch – OLED Model,OLED,Japan,FY3/2017,0.0
...,...,...,...,...,...
107,Nintendo Switch Lite,Lite,Other,FY3/2023,13
108,Software,Switch,Japan,FY3/2023,4579
109,Software,Switch,The Americas,FY3/2023,8700
110,Software,Switch,Europe,FY3/2023,6298


In [ ]:
df['dt'] = pd.to_datetime(df['Date'].str.replace('FY3/', '') + '-03-31')
df

,Product,Group,Geo,Date,Qty,dt
0,Nintendo Switch,Standard,Japan,FY3/2017,60.0,2017-03-31
1,Nintendo Switch,Standard,The Americas,FY3/2017,120.0,2017-03-31
2,Nintendo Switch,Standard,Europe,FY3/2017,76.0,2017-03-31
3,Nintendo Switch,Standard,Other,FY3/2017,18.0,2017-03-31
4,Nintendo Switch – OLED Model,OLED,Japan,FY3/2017,0.0,2017-03-31
...,...,...,...,...,...,...
107,Nintendo Switch Lite,Lite,Other,FY3/2023,13,2023-03-31
108,Software,Switch,Japan,FY3/2023,4579,2023-03-31
109,Software,Switch,The Americas,FY3/2023,8700,2023-03-31
110,Software,Switch,Europe,FY3/2023,6298,2023-03-31


In [ ]:
df = df.sort_values(["Group", "dt"], ascending=[True, True])

In [ ]:
geo_totals = df[~df.Product.isin(["Software"])].groupby(["dt", "Geo"])["Qty"].sum().reset_index()
fy_totals = df[~df.Product.isin(["Software"])].groupby(["dt"])["Qty"].sum().reset_index()
# create a dense rank of "dt" column over each "Group" value, ensure dt is sorted first in ascending order

df_models = df.copy()
filter_rows = True
filter_rows &= ~df_models.Product.isin(["Software"])
# filter_rows &= df_models.Qty > 0
df_models = df_models[filter_rows]
df_models["GroupRank"] = df_models.groupby(["Group"])["dt"].rank(method="dense", ascending=True)

model_totals = df_models.groupby(["Group", "GroupRank", "dt"])["Qty"].sum().reset_index()
model_totals

,Group,GroupRank,dt,Qty
0,Lite,1.0,2017-03-31,0.0
1,Lite,2.0,2018-03-31,0
2,Lite,3.0,2019-03-31,0
3,Lite,4.0,2020-03-31,620
4,Lite,5.0,2021-03-31,851
5,Lite,6.0,2022-03-31,369
6,Lite,7.0,2023-03-31,262
7,OLED,1.0,2017-03-31,0.0
8,OLED,2.0,2018-03-31,0
9,OLED,3.0,2019-03-31,0


In [ ]:
model_totals_chart = alt.Chart(model_totals).mark_line(point={"shape": "square", "size": 50}).encode(
    x=alt.X('dt:T'),
    y=alt.Y('Qty:Q'),
    color=alt.Color('Group:N')
)

fiscal_year_totals_chart = alt.Chart(fy_totals).mark_line(color='black', strokeDash=[5, 5], point={"shape": "square", "size": 50, "color": "black"}).encode(
    x=alt.X('dt:T'),
    y=alt.Y('Qty:Q'),
)


chart = fiscal_year_totals_chart + model_totals_chart
chart.interactive()

alt.LayerChart(...)